In [27]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import mean_absolute_error, mean_squared_error


In [28]:
X_train = np.load("/content/X_train.npy")
y_train = np.load("/content/y_train.npy")

X_val = np.load("/content/X_val.npy")
y_val = np.load("/content/y_val.npy")

print(X_train.shape, y_train.shape)


(14161, 30, 17) (14161,)


In [29]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)

X_val_t = torch.tensor(X_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.float32)


In [30]:
BATCH_SIZE = 64

train_loader = DataLoader(
    TensorDataset(X_train_t, y_train_t),
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_loader = DataLoader(
    TensorDataset(X_val_t, y_val_t),
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [31]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


In [32]:
class TransformerRUL(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2):
        super().__init__()

        self.input_projection = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )

        self.regressor = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_projection(x)
        x = self.pos_encoder(x)
        x = self.transformer(x)

        # Use last time step
        out = self.regressor(x[:, -1, :])
        return out.squeeze()


In [33]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = TransformerRUL(input_dim=X_train.shape[2]).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [34]:
EPOCHS = 25

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_preds, val_true = [], []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            preds = model(X_batch).cpu().numpy()
            val_preds.extend(preds)
            val_true.extend(y_batch.numpy())

    val_mae = mean_absolute_error(val_true, val_preds)
    val_rmse = np.sqrt(mean_squared_error(val_true, val_preds))

    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"Train MSE: {train_loss:.4f} | "
        f"Val MAE: {val_mae:.2f} | "
        f"Val RMSE: {val_rmse:.2f}"
    )


Epoch 1/25 | Train MSE: 11956.6477 | Val MAE: 85.83 | Val RMSE: 103.41
Epoch 2/25 | Train MSE: 11610.5664 | Val MAE: 84.14 | Val RMSE: 101.89
Epoch 3/25 | Train MSE: 11278.4503 | Val MAE: 82.22 | Val RMSE: 100.14
Epoch 4/25 | Train MSE: 10900.2849 | Val MAE: 80.16 | Val RMSE: 98.22
Epoch 5/25 | Train MSE: 10478.4768 | Val MAE: 77.74 | Val RMSE: 96.15
Epoch 6/25 | Train MSE: 10053.2064 | Val MAE: 75.26 | Val RMSE: 93.97
Epoch 7/25 | Train MSE: 9623.1613 | Val MAE: 72.70 | Val RMSE: 91.70
Epoch 8/25 | Train MSE: 9203.5996 | Val MAE: 70.13 | Val RMSE: 89.35
Epoch 9/25 | Train MSE: 8729.0940 | Val MAE: 67.49 | Val RMSE: 86.92
Epoch 10/25 | Train MSE: 8305.9082 | Val MAE: 64.88 | Val RMSE: 84.45
Epoch 11/25 | Train MSE: 7854.4444 | Val MAE: 62.27 | Val RMSE: 81.93
Epoch 12/25 | Train MSE: 7413.0952 | Val MAE: 59.59 | Val RMSE: 79.37
Epoch 13/25 | Train MSE: 6995.5359 | Val MAE: 57.02 | Val RMSE: 76.80
Epoch 14/25 | Train MSE: 6572.3412 | Val MAE: 54.49 | Val RMSE: 74.20
Epoch 15/25 | Train 

In [35]:
torch.save(model.state_dict(), "/content/transformer_rul_model.pth")
